In [2]:
# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
import numpy as np
import pandas as pd


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [3]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

2024-05-22 09:28:43.336397: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


### DNN Regression
It is now time to implements single-input and multiple-inputs DNN models

In [4]:
print('---------------------------------------------------------------------------------')
print('Initializing DNN program')
print('---------------------------------------------------------------------------------')
print('')
#---------------------------------------------------------------------
# Importing Libraries

print('---------------------------------------------------------------------------------')
print('Importing Libraries')
print('---------------------------------------------------------------------------------')
print('')

import time
import sklearn
import tensorflow as tf
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math
#import keras
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM
#from keras.layers import Dropout
#from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
#from keras.callbacks import EarlyStopping
#from keras.preprocessing import sequence
#from keras.utils import pad_sequences
#from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import shap
np.random.seed(0)

#---------------------------------------------------------------------
# Defining metric equations

print('---------------------------------------------------------------------------------')
print('Defining Metric Equations')
print('---------------------------------------------------------------------------------')
print('')

def ACC(TP,TN,FP,FN):
    Acc = (TP+TN)/(TP+FP+FN+TN)
    return Acc
def ACC_2 (TP, FN):
    ac = (TP/(TP+FN))
    return ac
def PRECISION(TP,FP):
    Precision = TP/(TP+FP)
    return Precision
def RECALL(TP,FN):
    Recall = TP/(TP+FN)
    return Recall
def F1(Recall, Precision):
    F1 = 2 * Recall * Precision / (Recall + Precision)
    return F1
def BACC(TP,TN,FP,FN):
    BACC =(TP/(TP+FN)+ TN/(TN+FP))*0.5
    return BACC
def MCC(TP,TN,FP,FN):
    MCC = (TN*TP-FN*FP)/(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**.5)
    return MCC
def AUC_ROC(y_test_bin,y_score):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    auc_avg = 0
    counting = 0
    for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
     # plt.plot(fpr[i], tpr[i], color='darkorange', lw=2)
      #print('AUC for Class {}: {}'.format(i+1, auc(fpr[i], tpr[i])))
      auc_avg += auc(fpr[i], tpr[i])
      counting = i+1
    return auc_avg/counting
#---------------------------------------------------------------------
# Defining features of interest
print('---------------------------------------------------------------------------------')
print('Defining features of interest')
print('---------------------------------------------------------------------------------')
print('')




---------------------------------------------------------------------------------
Initializing DNN program
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Importing Libraries
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Defining Metric Equations
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Defining features of interest
---------------------------------------------------------------------------------



/home/oarreche@ads.iu.edu/anaconda3/envs/tf23/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# First ensemble with NSL-KDD
# Parameters

#----------------------------------------------
# 0 for not using it as base learner
# 1 for using it as base learner

use_model_ada = 1 
use_model_dnn = 1 
use_model_mlp = 1 
use_model_lgbm = 1 
use_model_rf = 1 
use_model_svm = 1
use_model_knn = 1 
#----------------------------------------------
# 0 for training the model
# 1 for using the saved version of the model

load_model_ada = 0 
load_model_dnn = 0 
load_model_mlp = 0 
load_model_lgbm = 0 
load_model_rf = 0 
load_model_svm = 0
load_model_knn = 0 
#----------------------------------------------

# load_model_ada = 1
# load_model_dnn = 1 
# load_model_mlp = 1 
# load_model_lgbm = 1 
# load_model_rf = 1 
# load_model_svm = 1
# load_model_knn = 1 
#----------------------------------------------
feature_selection_bit = 0
# feature_selection_bit = 1



# Specify the name of the output text file
if feature_selection_bit == 0:
    output_file_name = "ensemble_base_models_all_features.txt"
    with open(output_file_name, "w") as f: print('---------------------------------------------------------------------------------', file = f)
    with open(output_file_name, "a") as f: print('---- ensemble_base_models_all_features', file = f)

elif feature_selection_bit == 1:
    output_file_name = "ensemble_base_models_feature_selection.txt"
    with open(output_file_name, "w") as f: print('---------------------------------------------------------------------------------', file = f)
    with open(output_file_name, "a") as f: print('----ensemble_base_models_feature_selection--', file = f)

#!/usr/bin/env python       
# coding: utf-8

# In[1]:
# importing required libraries
import numpy as np
import pandas as pd
import pickle # saving and loading trained model
from os import path


# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import (StandardScaler, OrdinalEncoder,LabelEncoder, MinMaxScaler, OneHotEncoder)
from sklearn.preprocessing import Normalizer, MaxAbsScaler , RobustScaler, PowerTransformer

# importing library for plotting
# import matplotlib.pyplot as plt
# import seaborn as sns

from sklearn import metrics
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# from keras.layers import Dense # importing dense layer

# from keras.layers import Input
# from keras.models import Model
# representation of model layers
#from keras.utils import plot_model
from sklearn.metrics import confusion_matrix
import shap

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
import time



In [6]:


start_program = time.time()



def confusion_metrics (name_model,predictions,true_labels,time_taken):

    name = name_model
    pred_label = predictions
    y_test_01 = true_labels 

    with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print(name, file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)
    FPR = FP / (FP + TN)
    FPR = 100*(sum(FPR)/len(FPR))

    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)


    # voting_acc_01 = Acc
    # voting_pre_01 = Precision
    # weighed_avg_rec_01 = Recall
    # weighed_avg_f1_01 = F1
    # weighed_avg_bacc_01 = BACC
    # weighed_avg_mcc_01 = MCC
    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)
    with open(output_file_name, "a") as f: print('Time Taken: ', time_taken, file = f)
    with open(output_file_name, "a") as f: print('FPR: ', FPR, '%' ,file = f)

    return Acc, Precision, Recall, F1, BACC, MCC, FPR




# In[3]:


# attach the column names to the dataset
feature=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", 
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]
# KDDTrain+_2.csv & KDDTest+_2.csv are the datafiles without the last column about the difficulty score
# these have already been removed.

train='KDDTrain+.txt'
test='KDDTest+.txt'

df=pd.read_csv(train,names=feature)
df_test=pd.read_csv(test,names=feature)



# shape, this gives the dimensions of the dataset
print('Dimensions of the Training set:',df.shape)
print('Dimensions of the Test set:',df_test.shape)


df.drop(['difficulty'],axis=1,inplace=True)
df_test.drop(['difficulty'],axis=1,inplace=True)



print('Label distribution Training set:')
print(df['label'].value_counts())
print()
print('Label distribution Test set:')
print(df_test['label'].value_counts())



# colums that are categorical and not binary yet: protocol_type (column 2), service (column 3), flag (column 4).
# explore categorical features
print('Training set:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

#see how distributed the feature service is, it is evenly distributed and therefore we need to make dummies for all.
print()
print('Distribution of categories in service:')
print(df['service'].value_counts().sort_values(ascending=False).head())



# Test set
print('Test set:')
for col_name in df_test.columns:
    if df_test[col_name].dtypes == 'object' :
        unique_cat = len(df_test[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))


from sklearn.preprocessing import LabelEncoder,OneHotEncoder
categorical_columns=['protocol_type', 'service', 'flag']
# insert code to get a list of categorical columns into a variable, categorical_columns
categorical_columns=['protocol_type', 'service', 'flag'] 
 # Get the categorical values into a 2D numpy array
df_categorical_values = df[categorical_columns]
testdf_categorical_values = df_test[categorical_columns]
df_categorical_values.head()


# protocol type
unique_protocol=sorted(df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]
# service
unique_service=sorted(df.service.unique())
string2 = 'service_'
unique_service2=[string2 + x for x in unique_service]
# flag
unique_flag=sorted(df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]
# put together
dumcols=unique_protocol2 + unique_service2 + unique_flag2
print(dumcols)

#do same for test set
unique_service_test=sorted(df_test.service.unique())
unique_service2_test=[string2 + x for x in unique_service_test]
testdumcols=unique_protocol2 + unique_service2_test + unique_flag2




df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)
print(df_categorical_values_enc.head())
# test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)



enc = OneHotEncoder()
df_categorical_values_encenc = enc.fit_transform(df_categorical_values_enc)
df_cat_data = pd.DataFrame(df_categorical_values_encenc.toarray(),columns=dumcols)
# test set
testdf_categorical_values_encenc = enc.fit_transform(testdf_categorical_values_enc)
testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc.toarray(),columns=testdumcols)

df_cat_data.head()


trainservice=df['service'].tolist()
testservice= df_test['service'].tolist()
difference=list(set(trainservice) - set(testservice))
string = 'service_'
difference=[string + x for x in difference]
difference

for col in difference:
    testdf_cat_data[col] = 0

testdf_cat_data.shape

newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)
# test data
newdf_test=df_test.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)
print(newdf.shape)
print(newdf_test.shape)


# take label column
labeldf=newdf['label']
labeldf_test=newdf_test['label']
# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
# put the new label column back
newdf['label'] = newlabeldf
newdf_test['label'] = newlabeldf_test
print(newdf['label'].head())


# Specify your selected features. Note that you'll need to modify this list according to your final processed dataframe
#Uncomment the below lines to use these top 20 features from shap analysis
#selected_features = ["root_shell","service_telnet","num_shells","service_uucp","dst_host_same_src_port_rate"
#                     ,"dst_host_rerror_rate","dst_host_srv_serror_rate","dst_host_srv_count","service_private","logged_in",
#                    "dst_host_serror_rate","serror_rate","srv_serror_rate","flag_S0","diff_srv_rate","dst_host_srv_diff_host_rate","num_file_creations","flag_RSTR"#,"dst_host_same_srv_rate","service_Idap","label"]
                     

# Select those features from your dataframe
#newdf = newdf[selected_features]
#newdf_test = newdf_test[selected_features]

# Now your dataframe only contains your selected features.

# creating a dataframe with multi-class labels (Dos,Probe,R2L,U2R,normal)
multi_data = newdf.copy()
multi_label = pd.DataFrame(multi_data.label)

multi_data_test=newdf_test.copy()
multi_label_test = pd.DataFrame(multi_data_test.label)


# using standard scaler for normalizing
std_scaler = StandardScaler()
def standardization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

numeric_col = multi_data.select_dtypes(include='number').columns
data = standardization(multi_data,numeric_col)
numeric_col_test = multi_data_test.select_dtypes(include='number').columns
data_test = standardization(multi_data_test,numeric_col_test)

# label encoding (0,1,2,3,4) multi-class labels (Dos,normal,Probe,R2L,U2R)
le2 = preprocessing.LabelEncoder()
le2_test = preprocessing.LabelEncoder()
enc_label = multi_label.apply(le2.fit_transform)
enc_label_test = multi_label_test.apply(le2_test.fit_transform)
multi_data = multi_data.copy()
multi_data_test = multi_data_test.copy()

multi_data['intrusion'] = enc_label
multi_data_test['intrusion'] = enc_label_test

#y_mul = multi_data['intrusion']
multi_data
multi_data_test



multi_data.drop(labels= [ 'label'], axis=1, inplace=True)
multi_data
multi_data_test.drop(labels= [ 'label'], axis=1, inplace=True)
multi_data_test


y_train_multi= multi_data[['intrusion']]
X_train_multi= multi_data.drop(labels=['intrusion'], axis=1)

print('X_train has shape:',X_train_multi.shape,'\ny_train has shape:',y_train_multi.shape)

y_test_multi= multi_data_test[['intrusion']]
X_test_multi= multi_data_test.drop(labels=['intrusion'], axis=1)

print('X_test has shape:',X_test_multi.shape,'\ny_test has shape:',y_test_multi.shape)


from collections import Counter

label_counts = Counter(y_train_multi['intrusion'])
print(label_counts)


from sklearn.preprocessing import LabelBinarizer

y_train_multi = LabelBinarizer().fit_transform(y_train_multi)

y_test_multi = LabelBinarizer().fit_transform(y_test_multi)


Y_train=y_train_multi.copy()
X_train=X_train_multi.copy()

Y_test=y_test_multi.copy()
X_test=X_test_multi.copy()


from imblearn.over_sampling import RandomOverSampler
from sklearn.datasets import make_classification

# Assuming you have features X and labels Y
# X, Y = make_classification()

ros = RandomOverSampler(sampling_strategy='minority', random_state=100)

X_train, Y_train = ros.fit_resample(X_train, Y_train)


# In[33]:


print(Y_test)


# In[34]:


X_train.values


# In[35]:
single_class_train = np.argmax(y_train_multi, axis=1)
single_class_test = np.argmax(y_test_multi, axis=1)


df1 = X_train_multi.assign(Label = single_class_train)
df2 =  X_test_multi.assign(Label = single_class_test)

frames = [df1,  df2]

df = pd.concat(frames,ignore_index=True)

# IG popping the features

# df.pop('same_srv_rate')
# df.pop('service_http')
# df.pop('flag_SF')
# df.pop('dst_host_count')
# df.pop('service_private')

# df.pop('logged_in')
# df.pop('count')
# df.pop('dst_host_serror_rate')
# df.pop('flag_S0')
# df.pop('Protocol_type_tcp')

# df.pop('dst_host_srv_serror_rate')
# df.pop('srv_diff_host_rate')
# df.pop('dst_host_same_srv_rate')
# df.pop('srv_serror_rate')
# df.pop('dst_host_srv_count')
# df.pop('serror_rate')
# df.pop('dst_host_same_src_port_rate')
# df.pop('Protocol_type_udp')
# df.pop('service_ecr_i')
# df.pop('srv_count')
# df.pop('srv_rerror_rate')

# df.pop('dst_host_srv_rerror_rate')
# df.pop('rerror_rate')
# df.pop('dst_host_srv_diff_host_rate')
# df.pop('diff_srv_rate')
# df.pop('service_other')
# df.pop('service_ftp_data')
# df.pop('service_smtp')
# df.pop('dst_host_rerror_rate')
# df.pop('dst_host_diff_srv_rate')
# df.pop('Protocol_type_icmp')
# df.pop('flag_REJ')
# df.pop('service_telnet')
# df.pop('hot')
# df.pop('service_eco_i')
# df.pop('flag_RSTR')
# df.pop('service_domain_u')
# df.pop('service_ftp')
# df.pop('flag_RSTO')
# df.pop('is_guest_login')
# df.pop('service_Z39_50')
# df.pop('duration')

# df.pop('service_iso_tsap')
# df.pop('service_pop_3')
# df.pop('service_ldap')
# df.pop('service_efs')
# df.pop('service_http_443')
# df.pop('service_daytime')
# df.pop('service_finger')
# df.pop('service_sunrpc')
# df.pop('service_systat')
# df.pop('service_echo')
# df.pop('service_discard')
# df.pop('service_uucp')
# df.pop('service_uucp_path')
# df.pop('service_mtp')
# df.pop('service_urp_i')
# df.pop('service_vmnet')
# df.pop('service_netbios_dgm')
# df.pop('service_courier')
# df.pop('service_name')
# df.pop('service_whois')
# df.pop('service_imap4')
# df.pop('service_csnet_ns')
# df.pop('service_login')
# df.pop('service_netstat')
# df.pop('service_kshell')
# df.pop('service_klogin')
# df.pop('service_netbios_ns')
# df.pop('service_bgp')
# df.pop('service_exec')
# df.pop('service_nnsp')
# df.pop('service_hostnames')
# df.pop('service_ctf')
# df.pop('service_link')
# df.pop('service_ssh')
# df.pop('wrong_fragment')
# df.pop('service_gopher')
# df.pop('service_sql_net')


# df.pop('service_netbios_ssn')
# df.pop('service_time')
# df.pop('flag_S1')
# df.pop('service_supdup')
# df.pop('flag_SH')
# df.pop('num_failed_logins')
# df.pop('service_domain')
# df.pop('root_shell')
# df.pop('dst_bytes')
# df.pop('num_access_files')
# df.pop('service_remote_job')
# df.pop('flag_RSTOS0')
# df.pop('service_auth')
# df.pop('service_shell')
# df.pop('service_pop_2')
# df.pop('service_IRC')
# df.pop('service_ntp_u')
# df.pop('flag_S3')
# df.pop('service_nntp')
# df.pop('flag_S2')
# df.pop('service_rje')
# df.pop('num_shells')
# df.pop('su_attempted')
# df.pop('service_printer')
# df.pop('num_file_creations')
# df.pop('service_X11')
# df.pop('src_bytes')
# df.pop('land')
# df.pop('flag_OTH')
# df.pop('num_root')
# df.pop('num_compromised')
# df.pop('service_tim_i')
# df.pop('service_pm_dump')
# df.pop('service_red_i')
# df.pop('urgent')
# df.pop('service_http_2784')
# df.pop('service_tftp_u')
# df.pop('is_host_login')
# df.pop('service_urh_i')
# df.pop('service_http_8001')
# df.pop('service_harvest')
# df.pop('service_aol')
# df.pop('num_outbound_cmds')



# LRP popping the features

# df.pop('flag_S0')
# df.pop('flag_SF')
# df.pop('srv_serror_rate')
# df.pop('dst_host_count')
# df.pop('dst_host_srv_count')

# df.pop('dst_host_srv_serror_rate') 
# df.pop('serror_rate')
# df.pop('service_http')
# df.pop('count')
# df.pop('dst_host_rerror_rate')

# df.pop('dst_host_same_srv_rate')
# df.pop('same_srv_rate')
# df.pop('logged_in')
# df.pop('Protocol_type_udp')
# df.pop('dst_host_serror_rate')
# df.pop('service_private')
# df.pop('service_ftp_data')
# df.pop('flag_REJ')
# df.pop('service_telnet')
# df.pop('dst_host_srv_diff_host_rate')
# df.pop('dst_host_srv_rerror_rate')

# df.pop('srv_count')
# df.pop('srv_rerror_rate')
# df.pop('service_eco_i')
# df.pop('dst_host_same_src_port_rate')
# df.pop('rerror_rate')
# df.pop('Protocol_type_icmp')
# df.pop('is_guest_login')
# df.pop('service_ftp')
# df.pop('service_iso_tsap')
# df.pop('service_domain_u')
# df.pop('service_uucp_path')
# df.pop('service_Z39_50')
# df.pop('dst_host_diff_srv_rate')
# df.pop('service_smtp')
# df.pop('diff_srv_rate')
# df.pop('service_vmnet')
# df.pop('service_courier')
# df.pop('Protocol_type_tcp')
# df.pop('service_bgp')
# df.pop('flag_RSTO')
# df.pop('srv_diff_host_rate')

# df.pop('service_other')
# df.pop('wrong_fragment')
# df.pop('duration')
# df.pop('service_ldap')
# df.pop('service_ecr_i')
# df.pop('service_login')
# df.pop('service_nntp')
# df.pop('flag_RSTR')
# df.pop('service_nnsp')
# df.pop('hot')
# df.pop('service_finger')
# df.pop('service_urp_i')
# df.pop('service_netbios_dgm')
# df.pop('service_efs')
# df.pop('service_pop_3')
# df.pop('service_uucp')
# df.pop('service_http_443')
# df.pop('service_auth')
# df.pop('flag_SH')
# df.pop('service_imap4')
# df.pop('service_gopher')
# df.pop('service_csnet_ns')
# df.pop('service_sunrpc')
# df.pop('service_klogin')
# df.pop('flag_S1')
# df.pop('service_IRC')
# df.pop('service_ctf')
# df.pop('service_domain')
# df.pop('service_time')
# df.pop('service_whois')
# df.pop('service_supdup')
# df.pop('service_hostnames')
# df.pop('num_failed_logins')
# df.pop('service_ntp_u')
# df.pop('flag_RSTOS0')
# df.pop('service_X11')
# df.pop('service_link')
# df.pop('service_mtp')
# df.pop('service_netstat')
# df.pop('service_discard')
# df.pop('service_pop_2')

# df.pop('service_echo')
# df.pop('service_exec')
# df.pop('service_kshell')
# df.pop('service_systat')
# df.pop('service_netbios_ns')
# df.pop('flag_S2')
# df.pop('service_netbios_ssn')
# df.pop('su_attempted')
# df.pop('num_access_files')
# df.pop('num_file_creations')
# df.pop('service_name')
# df.pop('service_remote_job')
# df.pop('flag_S3')
# df.pop('service_daytime')
# df.pop('service_rje')
# df.pop('num_shells')
# df.pop('root_shell')
# df.pop('service_shell')
# df.pop('service_ssh')
# df.pop('service_printer')
# df.pop('service_urh_i')
# df.pop('service_red_i')
# df.pop('land')
# df.pop('urgent')
# df.pop('service_sql_net')
# df.pop('num_root')
# df.pop('service_http_2784')
# df.pop('service_http_8001')
# df.pop('dst_bytes')
# df.pop('num_compromised')
# df.pop('src_bytes')
# df.pop('flag_OTH')
# df.pop('service_pm_dump')
# df.pop('service_tim_i')
# df.pop('service_tftp_u')
# df.pop('is_host_login')
# df.pop('service_harvest')
# df.pop('service_aol')
# df.pop('num_outbound_cmds')



# SHAP popping

# df.pop('serror_rate')
# df.pop('flag_S0')
# df.pop('dst_host_srv_serror_rate')
# df.pop('flag_SF')
# df.pop('srv_serror_rate')

# df.pop('dst_host_serror_rate')
# df.pop('dst_host_srv_count')
# df.pop('dst_host_same_srv_rate')
# df.pop('service_http')
# df.pop('dst_host_count')

# df.pop('same_srv_rate')
# df.pop('Protocol_type_udp')
# df.pop('dst_host_rerror_rate')
# df.pop('count')
# df.pop('service_ftp_data')
# df.pop('logged_in')
# df.pop('service_private')
# df.pop('dst_host_same_src_port_rate')
# df.pop('service_eco_i')
# df.pop('is_guest_login')

# df.pop('service_telnet')
# df.pop('dst_host_srv_diff_host_rate')
# df.pop('service_Z39_50')
# df.pop('service_iso_tsap')
# df.pop('service_ftp')
# df.pop('flag_REJ')
# df.pop('dst_host_diff_srv_rate')
# df.pop('srv_count')
# df.pop('dst_host_srv_rerror_rate')
# df.pop('diff_srv_rate')
# df.pop('service_pop_3')
# df.pop('service_domain_u')
# df.pop('Protocol_type_icmp')
# df.pop('service_other')
# df.pop('service_bgp')
# df.pop('service_nntp')
# df.pop('service_uucp_path')
# df.pop('service_courier')
# df.pop('rerror_rate')
# df.pop('Protocol_type_tcp')
# df.pop('flag_RSTR')

# df.pop('srv_diff_host_rate')
# df.pop('duration')
# df.pop('srv_rerror_rate')
# df.pop('service_ecr_i')
# df.pop('flag_RSTO')
# df.pop('service_vmnet')
# df.pop('service_smtp')
# df.pop('service_ldap')
# df.pop('service_urh_i')
# df.pop('service_login')
# df.pop('wrong_fragment')
# df.pop('hot')
# df.pop('service_finger')
# df.pop('service_netbios_dgm')
# df.pop('service_IRC')
# df.pop('flag_SH')
# df.pop('service_auth')
# df.pop('service_nnsp')
# df.pop('service_imap4')
# df.pop('service_klogin')
# df.pop('service_discard')
# df.pop('service_time')
# df.pop('service_urp_i')
# df.pop('service_sunrpc')
# df.pop('service_uucp')
# df.pop('service_echo')
# df.pop('land')
# df.pop('service_daytime')
# df.pop('service_mtp')
# df.pop('service_ctf')
# df.pop('service_sql_net')
# df.pop('service_gopher')
# df.pop('service_ntp_u')
# df.pop('service_supdup')
# df.pop('service_csnet_ns')
# df.pop('flag_S1')
# df.pop('flag_RSTOS0')
# df.pop('service_pop_2')
# df.pop('service_netbios_ns')
# df.pop('service_link')
# df.pop('service_http_443')
# df.pop('su_attempted')
# df.pop('flag_S2')

# df.pop('service_X11')
# df.pop('service_netstat')
# df.pop('num_failed_logins')
# df.pop('service_whois')
# df.pop('service_efs')
# df.pop('service_hostnames')
# df.pop('service_domain')
# df.pop('service_kshell')
# df.pop('service_red_i')
# df.pop('num_access_files')
# df.pop('flag_OTH')
# df.pop('service_shell')
# df.pop('root_shell')
# df.pop('service_printer')
# df.pop('service_systat')
# df.pop('service_ssh')
# df.pop('service_exec')
# df.pop('service_netbios_ssn')
# df.pop('service_tftp_u')
# df.pop('flag_S3')
# df.pop('service_rje')
# df.pop('service_name')
# df.pop('num_file_creations')
# df.pop('num_shells')
# df.pop('service_remote_job')
# df.pop('num_root')
# df.pop('num_compromised')
# df.pop('service_tim_i')
# df.pop('dst_bytes')
# df.pop('urgent')
# df.pop('src_bytes')
# df.pop('is_host_login')
# df.pop('service_pm_dump')
# df.pop('service_harvest')
# df.pop('service_http_8001')
# df.pop('service_aol')
# df.pop('service_http_2784')
# df.pop('num_outbound_cmds')



df_fs = df

y = df.pop('Label')
X = df
df = X.assign(Label = y)


# y = df.pop('Label')
# X = df

y1, y2 = pd.factorize(y)

y_0 = pd.DataFrame(y1)
y_1 = pd.DataFrame(y1)
y_2 = pd.DataFrame(y1)
y_3 = pd.DataFrame(y1)
y_4 = pd.DataFrame(y1)


# y_0 = y_0.replace(0, 0)
# y_0 = y_0.replace(1, 1)
y_0 = y_0.replace(2, 1)
y_0 = y_0.replace(3, 1)
y_0 = y_0.replace(4, 1)


y_1 = y_1.replace(1, 999)
y_1 = y_1.replace(0, 1)
# y_1 = y_1.replace(1, 0)
y_1 = y_1.replace(2, 1)
y_1 = y_1.replace(3, 1)
y_1 = y_1.replace(4, 1)
y_1 = y_1.replace(999, 1)


y_2 = y_2.replace(0, 1)
y_2 = y_2.replace(1, 1)
y_2 = y_2.replace(2, 0)
y_2 = y_2.replace(3, 1)
y_2 = y_2.replace(4, 1)


y_3 = y_3.replace(0, 1)
# y_3 = y_3.replace(1, 1)
y_3 = y_3.replace(2, 1)
y_3 = y_3.replace(3, 0)
y_3 = y_3.replace(4, 1)


y_4 = y_4.replace(0, 1)
# y_4 = y_4.replace(1, 1)
y_4 = y_4.replace(2, 1)
y_4 = y_4.replace(3, 1)
y_4 = y_4.replace(4, 0)



df = df.assign(Label = y)

#Divide the dataset between level 00 and level 01
import sklearn
from sklearn.model_selection import train_test_split
# split = 0.7 # 0.5

# # X_00,X_01, y_00, y_01 = sklearn.model_selection.train_test_split(X, y, train_size=split)
# X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=split)

Dimensions of the Training set: (125973, 43)
Dimensions of the Test set: (22544, 43)
Label distribution Training set:
normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: label, dtype: int64

Label distribution Test set:
normal             9711
neptune            4657
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess  

In [7]:

# Separate Training and Testing db
print('---------------------------------------------------------------------------------')
print('Separating Training and Testing db')
print('---------------------------------------------------------------------------------')
print('')

X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.7,random_state=42)
df = X.assign( Label = y)


---------------------------------------------------------------------------------
Separating Training and Testing db
---------------------------------------------------------------------------------



In [8]:
y_train

145897    1
77854     1
145078    0
118322    0
116812    0
         ..
119879    1
103694    1
131932    0
146867    3
121958    0
Name: Label, Length: 103961, dtype: int64

In [9]:
import numpy as np

import tensorflow as tf

# Now you can use Keras modules directly from tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten

# import keras
# from keras.models import Sequential
# from keras.layers import Dense, Flatten
import innvestigate
tf.compat.v1.disable_eager_execution()

In [10]:
from tensorflow.keras.callbacks import LearningRateScheduler

print('---------------------------------------------------------------------------------')
print('Defining the DNN model')
print('---------------------------------------------------------------------------------')
print('')

# dropout_rate = 0.01
# nodes = (len(X_train.columns,) + 5 //2) 
nodes = 5
print(nodes)

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(len(X_train.columns,))))

model.add(tf.keras.layers.Dense(nodes, activation='relu'))
# model.add(tf.keras.layers.Dense(nodes, activation='relu'))



# model.add(tf.keras.layers.Dense(nodes, activation='relu'))
# model.add(tf.keras.layers.Dense(nodes, activation='relu'))
# model.add(tf.keras.layers.Dense(nodes, activation='relu'))
# model.add(tf.keras.layers.Dense(nodes, activation='relu'))
# model.add(tf.keras.layers.Dense(nodes, activation='relu'))
model.add(tf.keras.layers.Dense(5))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()
#---------------------------------------------------------------------

# # Define your DNN model
# model = Sequential([
#     Flatten(input_shape=(28, 28)),  # Input layer
#     Dense(128, activation='relu'),  # Hidden layer
#     Dense(10, activation='relu')  # Output layer
# ])

# # Compile the model
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])



# # Save the trained model
# model.save('your_model.h5')

# # Load your trained model
# model = tf.keras.models.load_model('your_model.h5')

from tensorflow.keras.callbacks import EarlyStopping
start = time.time()

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)

# Modify model.fit to include the EarlyStopping callback
# model.fit(X_train, y_train, epochs=1000, batch_size=len(X_train), callbacks=[early_stopping,lr_scheduler])
model.fit(X_train, y_train, epochs=1000, batch_size=len(X_train), callbacks=[early_stopping])

# Pass the lr_scheduler callback to your model.fit() function
# model.fit(X_train, y_train, callbacks=[lr_scheduler], ...)


end = time.time()
print('---------------------------------------------------------------------------------')
print('ELAPSE TIME TRAINING MODEL: ',(end - start)/60, 'min')
print('---------------------------------------------------------------------------------')
print('')


---------------------------------------------------------------------------------
Defining the DNN model
---------------------------------------------------------------------------------

5


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 615       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 30        
Total params: 645
Trainable params: 645
Non-trainable params: 0
_________________________________________________________________
Train on 103961 samples


2024-05-22 09:28:52.039286: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-05-22 09:28:52.049355: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:67:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.54GiB deviceMemoryBandwidth: 715.34GiB/s
2024-05-22 09:28:52.049667: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:68:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-05-22 09:28:52.049728: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-05-22 09:28:52.085108: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-05

Epoch 1/1000
103961/103961 [==============================] - 0s 3us/sample - loss: 11.0542 - accuracy: 0.1158
Epoch 2/1000
103961/103961 [==============================] - 0s 2us/sample - loss: 1.7910 - accuracy: 0.7166
Epoch 3/1000
103961/103961 [==============================] - 0s 2us/sample - loss: 1.1850 - accuracy: 0.8111
Epoch 4/1000
103961/103961 [==============================] - 0s 2us/sample - loss: 1.0851 - accuracy: 0.8179
Epoch 5/1000
103961/103961 [==============================] - 0s 2us/sample - loss: 0.9580 - accuracy: 0.8278
Epoch 6/1000
103961/103961 [==============================] - 0s 2us/sample - loss: 1.3240 - accuracy: 0.8317
Epoch 7/1000
103961/103961 [==============================] - 0s 2us/sample - loss: 0.9235 - accuracy: 0.8311
Epoch 8/1000
103961/103961 [==============================] - 0s 2us/sample - loss: 0.9864 - accuracy: 0.8335
Epoch 9/1000
103961/103961 [==============================] - 0s 1us/sample - loss: 0.9786 - accuracy: 0.8335
Epoch 10/

In [11]:

print('---------------------------------------------------------------------------------')
print('Model Prediction')
print('---------------------------------------------------------------------------------')
print('')
print('---------------------------------------------------------------------------------')
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print('ELAPSE TIME MODEL PREDICTION: ',(end - start)/60, 'min')
print('---------------------------------------------------------------------------------')
print('')

#print(y_pred)
ynew = np.argmax(y_pred,axis = 1)
#print(ynew)
score = model.evaluate(X_test, y_test,verbose=1)
#print(score)
label = y2
# pred_label = label[ynew]
#print(score)

#---------------------------------------------------------------------
# pd.crosstab(test['ALERT'], preds, rownames=['Actual ALERT'], colnames = ['Predicted ALERT'])

from sklearn.metrics import accuracy_score

from collections import Counter

label_counts = Counter(y_test)
print(label_counts)

label_counts = Counter(ynew)
print(label_counts)

accuracy =accuracy_score(y_test, ynew)*100
print(accuracy)

---------------------------------------------------------------------------------
Model Prediction
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


ELAPSE TIME MODEL PREDICTION:  0.014004647731781006 min
---------------------------------------------------------------------------------

Counter({0: 23102, 1: 16081, 2: 4144, 3: 1197, 4: 32})
Counter({0: 25681, 1: 17694, 3: 1055, 2: 126})
86.6123529939851


In [12]:
single_sample_df = X_test.iloc[[0]]

## Integrated Gradients

In [30]:
%%time

# Create an analyzer for the model
analyzer = innvestigate.create_analyzer("integrated_gradients", model)

# Replace X_test with your input data

# X_test = np.random.rand(100, 28, 28)  # Example input data

# X_test2 = X_test.frac(0.001, random_state=42)

# Perform LRP analysis on the input data
analysis = analyzer.analyze(X_test)
#uncomment for single sample
# analysis = analyzer.analyze(single_sample_df)
# Print or use the analysis results as needed

# Perform LRP analysis on a certain number of samples
# analysis = analyzer.analyze(X_test.sample(1))

print(analysis)

print(len(X_test))
print(len(X_test.columns))
names = X_test.columns
print(analysis.shape)
print(type(analysis))
scores = pd.DataFrame(analysis)
print(analysis)
scores_abs = scores.abs()

# Calculate the sum of each column
sum_of_columns = scores_abs.sum(axis=0)

names = list(names)

names

sum_of_columns = list(sum_of_columns)

sum_of_columns
# type(sum_of_columns)
# # sorted_series = sum_of_columns.sort_values(ascending=False)

# print("Sorted Series in descending order:")
# print(sorted_series)
# names = ['John', 'Alice', 'Bob', 'Emily']
# sum_of_columns = [10, 5, 15, 8]

# Zip the two lists together
combined = list(zip(names, sum_of_columns))

# Sort the combined list in descending order based on the values from sum_of_columns
sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

# Unzip the sorted_combined list to separate names and sum_of_columns
sorted_names, sorted_sum_of_columns = zip(*sorted_combined)

print(sorted_names)
print(sorted_sum_of_columns)

sorted_names

[[-0.141 -0.004  0.001 -0.009 -0.177 -0.004 -0.03   0.049  0.586 -0.004
   0.008 -0.009 -0.003  0.01  -0.004  0.034  0.    -0.002  0.088  0.268
  -0.229  1.636  1.392 -0.549 -0.58   1.321 -0.012 -0.061  0.821  0.752
   0.73  -0.069  0.377 -0.026  1.576  1.666 -0.423 -0.416 -0.494  0.132
   0.302 -0.002  0.008 -0.018 -0.002  0.009 -0.03  -0.032 -0.032 -0.089
  -0.049 -0.038 -0.035  0.196 -0.04  -0.136 -0.21  -0.039  4.557 -0.108
   0.101 -0.169 -0.031 -0.001 -0.028  0.516 -0.    -0.062 -0.003 -0.054
  -0.047 -0.03  -0.027 -0.038 -0.032 -0.035 -0.034 -0.04  -0.038 -0.063
  -0.043 -0.032 -0.04  -0.035 -0.02   0.176 -0.005 -0.018  0.002 -0.024
  -0.335 -0.003 -0.02  -0.017 -0.018 -0.184 -0.027 -0.043 -0.053 -0.022
  -0.058 -0.014 -0.003 -0.005 -0.078 -0.001 -0.002 -0.05  -0.032 -0.023
  -0.033  0.    -0.281 -0.059 -0.027 -0.154  1.75   0.023 -0.024  0.01
   1.899 -0.061]]
44556
122
(1, 122)
<class 'numpy.ndarray'>
[[-0.141 -0.004  0.001 -0.009 -0.177 -0.004 -0.03   0.049  0.586 -0.004
   0

('service_exec',
 'flag_SF',
 'flag_S0',
 'dst_host_srv_serror_rate',
 'serror_rate',
 'dst_host_serror_rate',
 'srv_serror_rate',
 'same_srv_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'logged_in',
 'srv_rerror_rate',
 'rerror_rate',
 'service_http',
 'Protocol_type_icmp',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'dst_host_same_src_port_rate',
 'service_private',
 'Protocol_type_udp',
 'flag_REJ',
 'count',
 'srv_count',
 'service_ecr_i',
 'service_domain_u',
 'service_smtp',
 'wrong_fragment',
 'service_other',
 'service_ftp_data',
 'flag_RSTR',
 'duration',
 'service_eco_i',
 'Protocol_type_tcp',
 'service_finger',
 'service_ftp',
 'service_ctf',
 'is_guest_login',
 'service_time',
 'dst_host_diff_srv_rate',
 'service_netbios_ns',
 'service_http_443',
 'flag_SH',
 'srv_diff_host_rate',
 'flag_RSTO',
 'service_systat',
 'service_imap4',
 'service_sunrpc',
 'service_uucp',
 'num_failed_logins',
 'service_daytime',
 'service_iso_tsap',
 's

### lrp

## LRP

In [25]:
%%time

# Create an analyzer for the model
analyzer = innvestigate.create_analyzer("lrp.z", model)

# Replace X_test with your input data

# X_test = np.random.rand(100, 28, 28)  # Example input data

# X_test2 = X_test.frac(0.001, random_state=42)

# Perform LRP analysis on the input data
analysis = analyzer.analyze(X_test)
#uncomment for single sample
# analysis = analyzer.analyze(single_sample_df)

# Perform LRP analysis on a certain number of samples
# analysis = analyzer.analyze(X_test.sample(10000))

# Print or use the analysis results as needed
print(analysis)
print(len(X_test))
print(len(X_test.columns))
names = X_test.columns
print(analysis.shape)
print(type(analysis))
scores = pd.DataFrame(analysis)
print(analysis)
scores_abs = scores.abs()

# Calculate the sum of each column
sum_of_columns = scores_abs.sum(axis=0)

names = list(names)

names

sum_of_columns = list(sum_of_columns)

sum_of_columns
# type(sum_of_columns)
# # sorted_series = sum_of_columns.sort_values(ascending=False)

# print("Sorted Series in descending order:")
# print(sorted_series)
# names = ['John', 'Alice', 'Bob', 'Emily']
# sum_of_columns = [10, 5, 15, 8]

# Zip the two lists together
combined = list(zip(names, sum_of_columns))

# Sort the combined list in descending order based on the values from sum_of_columns
sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

# Unzip the sorted_combined list to separate names and sum_of_columns
sorted_names, sorted_sum_of_columns = zip(*sorted_combined)

print(sorted_names)
print(sorted_sum_of_columns)

sorted_names

[[-0.155 -0.007  0.004 -0.014 -0.188 -0.008 -0.013  0.016  0.613 -0.007
   0.008 -0.027 -0.01  -0.01  -0.013  0.011  0.    -0.003 -0.002  0.275
  -0.202 -0.782 -0.686  2.698  2.843  1.101 -0.019 -0.144  0.674  0.9
   0.819 -0.076  0.319 -0.037 -0.574 -0.685  2.358  2.543 -0.525 -0.06
  -0.032  0.008 -0.007 -0.057 -0.005 -0.023 -0.08  -0.077 -0.07  -0.078
  -0.083 -0.069 -0.068  0.032 -0.08  -0.146 -0.214 -0.073 -0.046 -0.106
  -0.    -0.063 -0.068 -0.004 -0.061  0.599 -0.002 -0.074 -0.005 -0.093
  -0.088 -0.069 -0.047 -0.078 -0.068 -0.071 -0.066 -0.064 -0.071 -0.068
  -0.075 -0.06  -0.08  -0.059 -0.024  0.004 -0.008 -0.031 -0.042 -0.03
   2.126 -0.009 -0.045 -0.03  -0.026 -0.203 -0.055 -0.073 -0.084 -0.058
  -0.067 -0.027 -0.006 -0.01  -0.057 -0.009 -0.024 -0.096 -0.085 -0.063
  -0.073 -0.014  2.413 -0.076 -0.046 -0.154 -0.624 -0.002 -0.037 -0.004
   1.381 -0.082]]
44556
122
(1, 122)
<class 'numpy.ndarray'>
[[-0.155 -0.007  0.004 -0.014 -0.188 -0.008 -0.013  0.016  0.613 -0.007
   0.00

('srv_rerror_rate',
 'rerror_rate',
 'dst_host_srv_rerror_rate',
 'flag_REJ',
 'dst_host_rerror_rate',
 'service_private',
 'flag_SF',
 'same_srv_rate',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'serror_rate',
 'srv_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_count',
 'flag_S0',
 'logged_in',
 'service_http',
 'dst_host_serror_rate',
 'Protocol_type_icmp',
 'dst_host_same_src_port_rate',
 'count',
 'service_ecr_i',
 'service_smtp',
 'srv_count',
 'wrong_fragment',
 'duration',
 'flag_RSTR',
 'service_eco_i',
 'srv_diff_host_rate',
 'service_finger',
 'service_uucp',
 'service_imap4',
 'service_iso_tsap',
 'service_uucp_path',
 'service_sunrpc',
 'service_daytime',
 'flag_SH',
 'service_bgp',
 'service_nnsp',
 'service_echo',
 'service_ldap',
 'service_ctf',
 'service_courier',
 'flag_RSTO',
 'dst_host_diff_srv_rate',
 'service_netbios_ssn',
 'service_http_443',
 'service_ssh',
 'service_efs',
 'service_whois',
 'service_netbios_dgm',
 'service_login',
 'service_csnet

In [15]:
# df.pop('flag_S0')
# df.pop('flag_SF')
# df.pop('srv_serror_rate')
# df.pop('dst_host_count')
# df.pop('dst_host_srv_count')
# df.pop('dst_host_srv_serror_rate') 
# df.pop('serror_rate')
# df.pop('service_http')
# df.pop('count')
# df.pop('dst_host_rerror_rate')
# df.pop('dst_host_same_srv_rate')
# df.pop('same_srv_rate')
# df.pop('logged_in')
# df.pop('Protocol_type_udp')
# df.pop('dst_host_serror_rate')
# df.pop('service_private')
# df.pop('service_ftp_data')
# df.pop('flag_REJ')
# df.pop('service_telnet')
# df.pop('dst_host_srv_diff_host_rate')
# df.pop('dst_host_srv_rerror_rate')
# df.pop('srv_count')
# df.pop('srv_rerror_rate')
# df.pop('service_eco_i')
# df.pop('dst_host_same_src_port_rate')
# df.pop('rerror_rate')
# df.pop('Protocol_type_icmp')
# df.pop('is_guest_login')
# df.pop('service_ftp')
# df.pop('service_iso_tsap')
# df.pop('service_domain_u')
# df.pop('service_uucp_path')
# df.pop('service_Z39_50')
# df.pop('dst_host_diff_srv_rate')
# df.pop('service_smtp')
# df.pop('diff_srv_rate')
# df.pop('service_vmnet')
# df.pop('service_courier')
# df.pop('Protocol_type_tcp')
# df.pop('service_bgp')
# df.pop('flag_RSTO')
# df.pop('srv_diff_host_rate')
# df.pop('service_other')
# df.pop('wrong_fragment')
# df.pop('duration')
# df.pop('service_ldap')
# df.pop('service_ecr_i')
# df.pop('service_login')
# df.pop('service_nntp')
# df.pop('flag_RSTR')
# df.pop('service_nnsp')
# df.pop('hot')
# df.pop('service_finger')
# df.pop('service_urp_i')
# df.pop('service_netbios_dgm')
# df.pop('service_efs')
# df.pop('service_pop_3')
# df.pop('service_uucp')
# df.pop('service_http_443')
# df.pop('service_auth')
# df.pop('flag_SH')
# df.pop('service_imap4')
# df.pop('service_gopher')
# df.pop('service_csnet_ns')
# df.pop('service_sunrpc')
# df.pop('service_klogin')
# df.pop('flag_S1')
# df.pop('service_IRC')
# df.pop('service_ctf')
# df.pop('service_domain')
# df.pop('service_time')
# df.pop('service_whois')
# df.pop('service_supdup')
# df.pop('service_hostnames')
# df.pop('num_failed_logins')
# df.pop('service_ntp_u')
# df.pop('flag_RSTOS0')
# df.pop('service_X11')
# df.pop('service_link')
# df.pop('service_mtp')
# df.pop('service_netstat')
# df.pop('service_discard')
# df.pop('service_pop_2')
# df.pop('service_echo')
# df.pop('service_exec')
# df.pop('service_kshell')
# df.pop('service_systat')
# df.pop('service_netbios_ns')
# df.pop('flag_S2')
# df.pop('service_netbios_ssn')
# df.pop('su_attempted')
# df.pop('num_access_files')
# df.pop('num_file_creations')
# df.pop('service_name')
# df.pop('service_remote_job')
# df.pop('flag_S3')
# df.pop('service_daytime')
# df.pop('service_rje')
# df.pop('num_shells')
# df.pop('root_shell')
# df.pop('service_shell')
# df.pop('service_ssh')
# df.pop('service_printer')
# df.pop('service_urh_i')
# df.pop('service_red_i')
# df.pop('land')
# df.pop('urgent')
# df.pop('service_sql_net')
# df.pop('num_root')
# df.pop('service_http_2784')
# df.pop('service_http_8001')
# df.pop('dst_bytes')
# df.pop('num_compromised')
# df.pop('src_bytes')
# df.pop('flag_OTH')
# df.pop('service_pm_dump')
# df.pop('service_tim_i')
# df.pop('service_tftp_u')
# df.pop('is_host_login')
# df.pop('service_harvest')
# df.pop('service_aol')
# df.pop('num_outbound_cmds')


## SHAP

In [20]:
%%time

output_file_name = 'SHAPCIC.txt'
with open(output_file_name, "w") as f:print('',file = f)

###here

print('---------------------------------------------------------------------------------')
print('Generating Explainer')
print('---------------------------------------------------------------------------------')

samples = 2500
# uncomment for single sample
# samples = 1

# Label = label
# df.pop('Label')
#train.pop('Label')
test = X_test
train = X_train
#df.pop('is_train')
start_index = 0

#end_index = len(test)
end_index = samples
#test = test[features]
explainer = shap.DeepExplainer(model,train[start_index:end_index].values.astype('float'))
shap_values = explainer.shap_values(test[start_index:end_index].values.astype('float'))
# shap_values = explainer.shap_values(test[start_index:len(test)].values.astype('float'))

# shap.summary_plot(shap_values = shap_values,
#                  features = test[start_index:end_index],
#                   class_names=[label[0],label[1],label[2],label[3],label[4],label[5],label[6]],show=False)

# plt.savefig('DNN_Shap_Summary_Cicids.png')
# plt.clf()

vals= np.abs(shap_values).mean(1)
feature_importance = pd.DataFrame(list(zip(train.columns, sum(vals))), columns=['col_name','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'], ascending=False,inplace=True)
feature_importance.head()
print(feature_importance.to_string())



print('---------------------------------------------------------------------------------')
# feature_importance_vals = 'feature_importance_vals'  # Replace with the name of the column you want to extract
feature_val = feature_importance['feature_importance_vals'].tolist()

# col_name = 'col_name'  # Replace with the name of the column you want to extract
feature_name = feature_importance['col_name'].tolist()


# for item1, item2 in zip(feature_name, feature_val):
#     print(item1, item2)


# Use zip to combine the two lists, sort based on list1, and then unzip them
zipped_lists = list(zip(feature_name, feature_val))
zipped_lists.sort(key=lambda x: x[1],reverse=True)

# Convert the sorted result back into separate lists
sorted_list1, sorted_list2 = [list(x) for x in zip(*zipped_lists)]

for k in sorted_list1:
  with open(output_file_name, "a") as f:print("df.pop('",k,"')", sep='',file = f)
with open(output_file_name, "a") as f:print("Trial_ =[", file = f)
for k in sorted_list1:
  with open(output_file_name, "a") as f:print("'",k,"',", sep='', file = f)
  print("'",k,"',", sep='')
with open(output_file_name, "a") as f:print("]", file = f)
print('---------------------------------------------------------------------------------')




# # shap.summary_plot(shap_values = shap_values[0],
# #                  features = test[start_index:end_index],
# #                   class_names=[label[0],label[1],label[2],label[3],label[4],label[5],label[6]],show=False)


# # plt.savefig('DNN_Shap_Summary_Beeswarms.png')
# # plt.clf()



# print('---------------------------------------------------------------------------------')
# print('Generating Sparsity Graph')
# print('---------------------------------------------------------------------------------')
# print('')

# # Find the minimum and maximum values in the list
# min_value = min(feature_val)
# max_value = max(feature_val)

# # Normalize the list to the range [0, 1]
# normalized_list = []
# for x in feature_val:
#     if max_value - min_value == 0:
#         normalized_list.append(0)
#     else:
#         normalized_list.append((x - min_value) / (max_value - min_value))
   
# # print(feature_name,normalized_list,'\n')
# # for item1, item2 in zip(feature_name, normalized_list):
# #     print(item1, item2)

# #calculating Sparsity

# # Define the threshold
# threshold = 1e-10

# # Initialize a count variable to keep track of values below the threshold
# count_below_threshold = 0

# # Iterate through the list and count values below the threshold
# for value in normalized_list:
#     if value < threshold:
#         count_below_threshold += 1

# Sparsity = count_below_threshold/len(normalized_list)
# Spar = []
# print('Sparsity = ',Sparsity)
# X_axis = []
# #----------------------------------------------------------------------------
# for i in range(0, 11):
#     i/10
#     threshold = i/10
#     for value in normalized_list:
#         if value < threshold:
#             count_below_threshold += 1

#     Sparsity = count_below_threshold/len(normalized_list)
#     Spar.append(Sparsity)
#     X_axis.append(i/10)
#     count_below_threshold = 0


# #---------------------------------------------------------------------------

# with open(output_file_name, "a") as f:print('y_axis_RF = ', Spar ,'', file = f)
# with open(output_file_name, "a") as f:print('x_axis_RF = ', X_axis ,'', file = f)

# plt.clf()

# # Create a plot
# plt.plot(X_axis, Spar, marker='o', linestyle='-')

# # Set labels for the axes
# plt.xlabel('X-Axis')
# plt.ylabel('Y-Axis')

# # Set the title of the plot
# plt.title('Values vs. X-Axis')

# # Show the plot
# # plt.show()
# plt.savefig('sparsity.png')
# plt.clf()

---------------------------------------------------------------------------------
Generating Explainer
---------------------------------------------------------------------------------


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


                        col_name  feature_importance_vals
95                  service_smtp                28.104786
53              service_domain_u                17.931668
39             Protocol_type_tcp                14.530624
40             Protocol_type_udp                14.300964
29            dst_host_srv_count                12.382734
8                      logged_in                12.088426
27            srv_diff_host_rate                 6.698434
30        dst_host_same_srv_rate                 5.150577
36          dst_host_rerror_rate                 4.789332
35      dst_host_srv_serror_rate                 1.443021
116                      flag_S0                 1.188646
31        dst_host_diff_srv_rate                 1.147201
34          dst_host_serror_rate                 1.115802
88                 service_pop_3                 1.053394
22               srv_serror_rate                 1.045836
21                   serror_rate                 1.041021
24            